In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [2]:
# COMMON
# env = gym.make("CartPole-v1")
env = gym.make("LunarLander-v3")
device = "cpu"
BATCH_SIZE = 128
HIDDEN_DIM = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 5000
TAU = 0.005
LR = 3e-4
MEMORY_SIZE = int(1E5)

# MY IMPLEMENTATION
TRAIN_EVERY = 1

c:\Users\Clément\Desktop\RainbowTorch\venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


## Common implementation

In [3]:

class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer = torch.nn.Sequential(
        torch.nn.Linear(n_observations, HIDDEN_DIM), torch.nn.ReLU(),
        torch.nn.Linear(HIDDEN_DIM, HIDDEN_DIM), torch.nn.ReLU(),
        torch.nn.Linear(HIDDEN_DIM, HIDDEN_DIM), torch.nn.ReLU(),
        torch.nn.Linear(HIDDEN_DIM, n_actions)
    )

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        return self.layer(x)

## Set up torch implementation

In [4]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


In [ ]:


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    




# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer




# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(MEMORY_SIZE)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


metric_to_plot = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(metric_to_plot, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

layer.0.weight Parameter containing:
tensor([[-0.0105, -0.0834,  0.2556,  ..., -0.0589,  0.1923, -0.0328],
        [-0.1855,  0.2942, -0.0101,  ...,  0.2289,  0.0792, -0.2281],
        [-0.0905, -0.2371,  0.3186,  ..., -0.3509,  0.3200, -0.0291],
        ...,
        [-0.1738, -0.2934, -0.1607,  ...,  0.2083,  0.0533,  0.2644],
        [-0.3118, -0.1897, -0.2179,  ...,  0.3258, -0.1586, -0.1711],
        [-0.1981, -0.1624,  0.1675,  ...,  0.3465, -0.0034, -0.1492]],
       requires_grad=True)
layer.0.bias Parameter containing:
tensor([ 0.3034,  0.1843,  0.2829,  0.3159, -0.0299,  0.1589,  0.2006,  0.1392,
         0.3504, -0.3060, -0.2733,  0.3107, -0.2974,  0.1568,  0.2277,  0.1744,
         0.0953,  0.3125, -0.3295,  0.1196,  0.3253, -0.0090,  0.1299, -0.3176,
        -0.1753, -0.2778,  0.2537,  0.2408, -0.2342,  0.0569,  0.0307, -0.1752,
         0.1397,  0.1097, -0.2283, -0.1170,  0.2929,  0.0330, -0.1557,  0.1814,
         0.1345, -0.2691,  0.1081,  0.0055,  0.1841,  0.2820,  0.29

In [6]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()
    return loss.item()

In [7]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 300

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    episode_rewards = 0
    episode_losses = []
    episode_steps = 0

    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        episode_rewards += reward
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        loss = optimize_model()

        episode_steps += 1
        if loss is not None : episode_losses.append(loss)

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            metric_to_plot.append(episode_rewards)
            epsilon = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
            episode_loss = np.array(episode_losses).mean()
            print(f"Episode {i_episode:3d} - Steps : {episode_steps:4d} | Total Rewards : {episode_rewards:7.2f} | Loss : {episode_loss:0.2e} | Epsilon : {epsilon : 0.2f} | Agent Step : {steps_done}")
            # print(episode_losses)
            # plot_durations()
            break

# print('Complete')
# plot_durations(show_result=True)
# plt.ioff()
# plt.show()

Episode   0 - Steps :  103 | Total Rewards : -149.93 | Loss : nan | Epsilon :  0.88 | Agent Step : 103


C:\Users\Clément\AppData\Local\Temp\ipykernel_12364\3034668483.py:49: RuntimeWarning: Mean of empty slice.
  episode_loss = np.array(episode_losses).mean()
c:\Users\Clément\Desktop\RainbowTorch\venv\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Episode   1 - Steps :   94 | Total Rewards :  -95.88 | Loss : 1.82e+00 | Epsilon :  0.87 | Agent Step : 197
Episode   2 - Steps :   96 | Total Rewards :   17.08 | Loss : 1.71e+00 | Epsilon :  0.85 | Agent Step : 293
Episode   3 - Steps :  131 | Total Rewards : -170.59 | Loss : 1.91e+00 | Epsilon :  0.83 | Agent Step : 424
Episode   4 - Steps :  113 | Total Rewards : -266.90 | Loss : 1.47e+00 | Epsilon :  0.81 | Agent Step : 537
Episode   5 - Steps :   96 | Total Rewards :  -53.20 | Loss : 1.58e+00 | Epsilon :  0.80 | Agent Step : 633
Episode   6 - Steps :   96 | Total Rewards :    4.72 | Loss : 1.62e+00 | Epsilon :  0.78 | Agent Step : 729
Episode   7 - Steps :  114 | Total Rewards : -146.69 | Loss : 1.73e+00 | Epsilon :  0.77 | Agent Step : 843
Episode   8 - Steps :   79 | Total Rewards :  -59.77 | Loss : 1.63e+00 | Epsilon :  0.76 | Agent Step : 922
Episode   9 - Steps :  104 | Total Rewards :  -75.78 | Loss : 1.82e+00 | Epsilon :  0.74 | Agent Step : 1026
Episode  10 - Steps :  104 

KeyboardInterrupt: 